In [17]:
import gradio as gr
from langgraph.prebuilt import create_react_agent

In [18]:
def mortgage_payment_calculator(loan_amount: float, annual_interest_rate: float, number_of_payments:int):
    """Calculate the monthly payment of a mortgage"""
    monthly_interest_rate=annual_interest_rate/12
    monthly_payment=loan_amount*monthly_interest_rate*(1 + monthly_interest_rate)**number_of_payments/\
                    ((1.0 + monthly_interest_rate)**number_of_payments - 1.0)
    return monthly_payment

In [11]:
mortgage_payment_calculator(50000.0,0.06,60)

966.6400764714135

In [29]:
mortgage_agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[mortgage_payment_calculator],
    prompt=(
        "You are a mortgage agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with mortgage payment calculation\n"
        "- Ask for loan amount, annual interest rate, or number of payments, if any of them not provided\n"
        "- Combine already provided function parameters in history to complete the calculation\n" 
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="mortgage_agent",
)

In [40]:
from langchain.schema import AIMessage, HumanMessage  
async def chat_with_agent(message, history):
    # LangGraph agents expect messages in a specific format
    #print(history)
    messages = history + [{"role": "user", "content": message}]    
    # Invoke the agent
    response = await mortgage_agent.ainvoke({"messages": messages})
    
    # Extract the agent's reply
    agent_reply = response["messages"][-1].content
    return agent_reply

# 5. Create the Gradio interface
iface = gr.ChatInterface(
    fn=chat_with_agent,
    type="messages"
)

# 6. Launch the Gradio app
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.
